В этом ноутбуке кортежи мнений предсказываются с помощью модели Grok-2 (grok-beta).

Здесь используется OpenAI API, также необходим XAI API key.

Рекомендуется запускать из Google Colab, а не локально, во избежание геополитических ограничений.

### Проверка ###

In [3]:
import requests
import ast
import json
import os
import itertools
from tqdm import tqdm
import pandas as pd
from collections import defaultdict
import random
from random import choices
import time

from openai import OpenAI

XAI_API_KEY = "YOUR_API_KEY"
model = "grok-beta"
url = "https://api.x.ai/v1"

SEED = 42
random.seed(SEED)

client = OpenAI(
    api_key=XAI_API_KEY,
    base_url=url,
)

params = {'temperature': 0.2,
         'top_p': 0.9,
         'max_tokens': 512,
         'seed': SEED}

completion = client.chat.completions.create(
    model=model,
    **params,
    messages=[
        {"role": "system", "content": "Ответь на вопрос."},
        {"role": "user", "content": "Тебя сделал Илон Маск?"},
    ],
)

print(completion.choices[0].message.content)

Нет, меня создала компания xAI. Я Грок, приятно познакомиться! Илон Маск - это крутой парень, но у меня другие создатели.


### Подготовка данных ###

In [4]:
import requests, zipfile, io

url = 'https://raw.githubusercontent.com/rossyaykin/RuOpinionNE/refs/heads/main/src/src.zip'
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall('')

from src.utils import parse_data, to_jsonl, form_prompt, dict2tuple, extract_tuple, str2list, save_jsonl, df2structure

In [5]:
!wget https://raw.githubusercontent.com/dialogue-evaluation/RuOpinionNE-2024/refs/heads/master/train.jsonl

--2024-11-27 01:16:31--  https://raw.githubusercontent.com/dialogue-evaluation/RuOpinionNE-2024/refs/heads/master/train.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1291979 (1.2M) [text/plain]
Saving to: ‘train.jsonl’

train.jsonl         100%[===================>]   1.23M  --.-KB/s    in 0.05s   

2024-11-27 01:16:31 (23.2 MB/s) - ‘train.jsonl’ saved [1291979/1291979]



In [6]:
!wget https://raw.githubusercontent.com/dialogue-evaluation/RuOpinionNE-2024/refs/heads/master/validation.jsonl

--2024-11-27 01:17:27--  https://raw.githubusercontent.com/dialogue-evaluation/RuOpinionNE-2024/refs/heads/master/validation.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 368114 (359K) [text/plain]
Saving to: ‘validation.jsonl’

validation.jsonl    100%[===================>] 359.49K  --.-KB/s    in 0.03s   

2024-11-27 01:17:28 (12.6 MB/s) - ‘validation.jsonl’ saved [368114/368114]



In [7]:
train = parse_data('/content/train.jsonl')
val = parse_data('/content/validation.jsonl')
print(len(train), len(val))

2556 1316


In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')
# train = parse_data('/content/gdrive/My Drive/train.jsonl')
# val = parse_data('/content/gdrive/My Drive/validation.jsonl')
# print(len(train), len(val))

2556 1316


### Определения ###

In [8]:
class Runner():
    def __init__(self, model, client, params, train, test, n_shots = 5, sleeptime = 2):
        self.model = model
        self.client = client
        self.params = params
        self.train = train
        self.test = test
        self.n_shots = n_shots
        self.sleeptime = sleeptime

    def run(self):
        results = list()
        for entry in tqdm(self.test):
            time.sleep(self.sleeptime)
            examples = [dict2tuple(x) for x in choices(self.train, k = n_shots)]
            prompt = form_prompt(examples, entry['text'])
            completion = client.chat.completions.create(model=self.model,
                                                        **self.params,
                                                        messages=[{"role": "user", "content": prompt}],)
            result = completion.choices[0].message.content
            try:
                result = ast.literal_eval(result)
            except (SyntaxError, ValueError):
                print(f'bad response, iteration:{len(results)}')
                result = []
            results.append((entry['sent_id'],
                            entry['text'],
                            dict2tuple(entry)[1], # gold opinions
                            result)) # pred opinions
        return results

def get_path(temp, n_shots):
    path = f'/content/Grok_bl_{n_shots}shot_{temp}temp'
    # returns full path but without ".csv"
    return path

def save(dataframe, path, raw = True):
    outdir, outname = '/'.join(path.split('/')[:-1]), path.split('/')[-1]
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    if raw:
        dataframe.to_csv(f'{path}_raw.csv', index = False)
    else:
        dataframe.to_csv(f'{path}.csv', index = False)

### Тест на одном примере ###

In [9]:
n_shots = 5
examples = [dict2tuple(x) for x in train[:n_shots]]
text, target = dict2tuple(train[n_shots])

sample_prompt = form_prompt(examples, text)
print(f'--Промпт--:\n\n{sample_prompt}')

--Промпт--:

Ты эксперт в оценке тональности.
Тебе нужно найти все негативные и позитивные отношения между сущностями в тексте и вывести их в следующем формате:
[источник отношения, объект отношения, выражение в тексте содержащее оценку, оценка (POS/NEG)]
Если источником отношения является автор, то пиши:
['AUTHOR', объект отношения, выражение в тексте содержащее оценку, оценка (POS/NEG)]
Если выраженного источника нет, то пиши:
['NULL', объект отношения, выражение в тексте содержащее оценку, оценка (POS/NEG)]
Допустимо вернуть пустой ответ:
[]
Не нужно давать пояснений к ответу.
Примеры
Текст: Президент Башкирии Муртаза Рахимов в очередной раз решил поменять главу своей администрации.
Ответ: [['Муртаза Рахимов', 'главу своей администрации', 'поменять', 'NEG']]
Текст: Вчера он уволил Азамата Сагитова, который возглавил башкирскую администрацию год назад после вынужденной отставки Радия Хабирова, сейчас занимающего пост заместителя начальника управления президента РФ по внутренней полит

In [10]:
completion = client.chat.completions.create(
    model=model,
    **params,
    messages=[{"role": "user",
               "content": sample_prompt}],
)

response = completion.choices[0].message.content
result = ast.literal_eval(response)

print(f'--Текст--:\n{text}')
print(f'--Таргет--:\n{target}')
print(f'--Предикт--:\n{extract_tuple(result)}')

--Текст--:
Этому назначению предшествовал громкий скандал, сопровождавший историю отставки прежнего главы администрации Радия Хабирова.
--Таргет--:
[['NULL', 'Радия Хабирова', 'громкий скандал', 'NEG']]
--Предикт--:
[['NULL', 'скандал', 'громкий', 'NEG']]


### Инференс ###

In [ ]:
%%time
params = {'temperature': 0.2,
          'top_p': 0.9,
          'max_tokens': 512,
         'seed': SEED}
n_shots = 11
runner = Runner(model, client, params, train, val, n_shots)

path = get_path(params['temperature'], n_shots)
result = runner.run()

100%|██████████| 1316/1316 [59:33<00:00,  2.72s/it]

CPU times: user 26.5 s, sys: 2.63 s, total: 29.1 s
Wall time: 59min 33s


In [ ]:
print(path)
print(result[0])

/content/gdrive/My Drive/Grok_bl_11shot_0.2temp
(0, 'В числе участников президентской борьбы есть одна женщина — Айссата Хайдара Сиссе (Aissata Haidara Cisse).', [], [])


### Результаты ###

In [ ]:
output = pd.DataFrame(result, columns = ['sent_id', 'text', 'target', 'pred'])
output.head()

,sent_id,text,target,pred
0,0,В числе участников президентской борьбы есть о...,[],[]
1,1,"Кандидатке на пост президента 54 года, она род...",[],[]
2,2,"Сама женщина заявила, что встречаться с сыном ...",[],"[[сама женщина, сыном, пришлось в присутствии ..."
3,3,"Они снимали нас все эти 5 минут, что длилось с...",[],[]
4,4,"Кроме того, по словам женщины на щеке сына Све...",[],"[[Светлана, сына, синяк, NEG]]"


In [ ]:
print(path)
save(output, path)

/content/gdrive/My Drive/Grok_bl_11shot_0.2temp


In [ ]:
output = pd.DataFrame([(x[0], x[1], x[2], str2list(extract_tuple(x[3]))) for x in result],
                      columns = ['sent_id', 'text', 'target', 'pred'])
output.head()

,sent_id,text,target,pred
0,0,В числе участников президентской борьбы есть о...,[],[]
1,1,"Кандидатке на пост президента 54 года, она род...",[],[]
2,2,"Сама женщина заявила, что встречаться с сыном ...",[],"[[сама женщина, сыном, пришлось в присутствии ..."
3,3,"Они снимали нас все эти 5 минут, что длилось с...",[],[]
4,4,"Кроме того, по словам женщины на щеке сына Све...",[],"[[Светлана, сына, синяк, NEG]]"


In [ ]:
output['pred'].value_counts()[0]

<ipython-input-11-c97be47da270>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  output['pred'].value_counts()[0]


536

In [ ]:
save(output, path, raw = False)

### csv to jsonl ###

In [ ]:
final = df2structure(output)
final[3]

{'sent_id': 3,
 'text': 'Они снимали нас все эти 5 минут, что длилось свидание, чтобы Влад ничего лишнего мне не сказал.',
 'opinions': []}

In [ ]:
save_jsonl(final, path)